# Trabajo Final LP2

## Integrantes:

- Poma Gómez Diego Alonso
- Kuway Chero Giancarlo Javier
- Manuel Mercedez

In [68]:
################################################# PARA LIBRERIAS #################################################

#pip install pretty-html-table
#pip install selenium

# Descargar

In [ ]:
################################################# PARA INGRESAR CORREO Y PRODUCTO #################################################


# Ingresa producto y correo electrónico
producto= input("Ingrese el producto a comparar: ")
correo= input("Ingrese su dirección de correo electrónico (ej: example@gmail) ")
ubicacion='C:/Users/gian_/OneDrive/Escritorio/Trabajo_Final_LP2/chromedriver.exe'

################################################# PARA METRO #################################################

# Se importan librerias
from selenium.webdriver.common.keys import Keys                  # Para cambiar el nombre de un producto inválido ingresado
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import random
from time import sleep
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(ubicacion,options = options)

# Se asigna la pagina web a scrapear
driver.get("https://www.metro.pe/") 

# Da click para aceptar términos de un aviso emergente
WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'button.swal2-confirm swal2-styled'.replace(' ','.')))).click()

# Selecciona y escribe el producto en el buscador Metro 
WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'input#search-autocomplete-input'))).send_keys(producto)


# Se verifica si el nombre del producto ingresado es válido

while True:
    try:
        # Click en 'Ver todos los resultados'
        WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR,'a.see-all'.replace(' ','.')))).click()
        break
    except:
        print("No se encontraron resultados, nombre de producto inválido")
        producto=input('Vuelva a ingresar el nombre de producto: ')
        # Se sombrea el texto del producto inválido
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#search-autocomplete-input'))).send_keys(Keys.CONTROL + 'a')
        sleep(1)
        # Se escribe el nuevo nombre de producto ingresado
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#search-autocomplete-input'))).send_keys(producto)

# Hace scroll directo hacia el final de página
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")     
sleep(2)
# Espera a que carguen todos los productos de la página
WebDriverWait(driver, 120).until(
    EC.presence_of_all_elements_located((
        By.XPATH,'//div[@class="shelf-content"]/div[@style="display: none;"]')))  

elementos= driver.find_elements(By.XPATH,'//li[@layout="19ccd66b-b568-43cb-a106-b52f9796f5cd"]')

datos1=[]
# Se extraen el nombre(junto con la cantidad) y el precio
for elem in elementos:
    saco=[]
    try:
        precio= elem.find_element("xpath",'.//span[@class="product-prices__value product-prices__value--best-price"]').text
        nombre= elem.find_element("xpath",'.//a[@class="product-item__name"]').text
        #print(nombre)
        #print(precio)
        saco.append(nombre)
        saco.append(precio)
        datos1.append(saco)
    except:
        precio = 'NO DISPONIBLE'
        producto = 'NO DISPONIBLE'
        
# LIMPIEZA Y ESTRUCTURACIÓN DE DATOS DESCARGADOS (METRO)

# Se crea la columna Cantidad
# Se importan librerias
import re
import pandas as pd

total1=[]

for i in datos1:
    saco1=[]   # nombre-precio-cantidad
    saco2=[]   # cantidad
    saco3=[]   # Agrupa los datos de cada producto
    saco1.append(i[0])
    for e in i:
        if e==i[0]:
            x= re.findall('\sx\s',e)    # Posee letra _x_ ?
            y= re.findall('[Pp]ack',e) # Posee palabra P-pack en el texto ?
            z= re.findall('^Mix',e)     # Posee letra Mix al comienzo ?
            w=re.findall('\sx.+',e)     # Coincide desde x_ hasta el final de la oración
            o= re.findall('\s[/]\s',e)  # Posee símbolo '/' 
            u= re.findall('\s[+]\s',e)  # Posee símbolo '+'
            q=re.findall('\d.+',e)      # Coincide con cualquier numero_ hasta el final de la oración
            oq=re.findall('\d+',e)      # Coincide con todos los números enteros
            if len(y)>0:
                saco2.append('Pack') #se almacena pack como cantidad
            elif len(z)>0:
                saco2.append('Mix') # se almacena Mix como cantidad

            elif len(x)>0:
                if len(w)>0:
                    saco2.append(w[0])

            elif len(x)==0:
                if len(oq)==0:
                    if len(u)>0:
                        saco2.append('Duo')
                    else:
                        saco2.append('1 unidad')
                elif len(o)>0:
                    saco2.append(q[0])
                elif len(u)==0:
                    v=re.findall('\d+\s+([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)',e)
                    v1=re.findall('\d+([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)',e)
                    # v y v1 sirven para encontrar condiciones en la oracion 'e'
                    if len(oq)>=2:
                        t2=re.findall('%',e)
                        lp=e.split()    # Separa la oración en palabras
                        a=0     # contador
                        for elem in lp:
                            t=re.findall('^\d.+([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)$',elem)
                            t1=re.findall('^([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)$',elem)

                            if len(t)==1:
                                saco2.append(lp[a])              # Si coincide, agregar a saco2
                            elif len(t1)==1:        
                                saco2.append(lp[a-1]+' '+lp[a])  # de lo contrario, juntarlo con el elemento anterior y agregarlo
                            a=a+1                                # a saco2
                    else:
                        saco2.append(q[0])
                        
                    if len(v)==0 or len(v1)==0:
                        saco2.append(q[0])
                    
                elif len(u)>0:
                    saco2.append('Pack')
            else:
                saco2.append(q[0])
            
            if saco2[0]=='Pack' or saco2[0]=='Mix':
                saco3.append('Metro')
                saco3.append(saco1[0])
            else:
                saco1=saco1[0].replace(saco2[0],'')
                saco3.append('Metro')
                saco3.append(saco1)
                
        elif e==i[1]:
            e=e.split()
            for q in e:
                z=re.findall('\d.+',q)
                if len(z)>0:
                    saco3.append(z[0])
            
    saco3.append(saco2[0])
    total1.append(saco3)        

## Longitud de los datos
# len(total1

# Se crea dataframe para estructurar datos con las columnas Nombre-Precio(S/.)-Cantidad

columnas=['Local','Nombre','Precio (S/.)','Cantidad']
Metro=pd.DataFrame(data=total1,columns=columnas)           

driver.close()
#Metro
print("Datos de Metro descargados")
################################################# PARA TOTTUS #################################################

# Se importan librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import random
from time import sleep
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument('--incognito')

driver = webdriver.Chrome(ubicacion,options = options)

# Se asigna la pagina web a scrapear
driver.get("https://www.tottus.com.pe/") 

# Selecciona y escribe el producto en el buscador Tottus 
try:
    WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#catalystSearchBar'))).click()
    WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#inputSearchBar'))).send_keys(producto)
    #sleep(1)
    WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'button.jsx-2851002401 searchIcon'.replace(' ','.')))).click()
except:
    WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input.jsx-3290144217 desktopSearchBar'.replace(' ','.')))).send_keys(producto)
    WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'button.jsx-3290144217 searchIcon'.replace(' ','.')))).click()


datos3=[]

while True:
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,'//li[@class="product"]')))
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
    sleep(1)

    elementos=driver.find_elements(By.XPATH,'//li[@class="product"]')
    for elem in elementos:
        saco=[]
        try:
            nombre= elem.find_element("xpath",'.//h2[@class="jsx-369422875 name title mini"]').text
            precio= elem.find_element("xpath",'.//span[@class="list price medium cmrPrice "]').text
            cantidad= elem.find_element("xpath",'.//h3[@class="jsx-369422875 extra text descriptive"]').text
            #print(nombre)
            #print(precio)
            #print(cantidad)
            saco.append(nombre)
            saco.append(precio)
            saco.append(cantidad)
            datos3.append(saco)
        except:
            precio = 'NO DISPONIBLE'
            producto = 'NO DISPONIBLE'

    driver.execute_script("window.scrollBy(0, -800);")
    sleep(1)
    try:
        WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((By.XPATH,'//li[@class="next"]'))).click()
        sleep(2)
    except:
        break
        

# LIMPIEZA Y ESTRUCTURACIÓN DE DATOS DESCARGADOS (Tottus)

# Se importan librerias
import re
import pandas as pd

total3=[]

for i in datos3:
    saco1=[]   # nombre-precio-cantidad
    saco1.append('Tottus')
    for e in i:
        if e==i[0]:
            saco1.append(e)
        if e==i[1]:
            e=e.split()
            for q in e:
                z=re.findall('\d.+',q)
                z1=re.findall('\d+',q)
                if len(z)>0: 
                    saco1.append(z[0])
                elif len(z1)>0:
                    saco1.append(z1[0])
        if e==i[2]:
            saco1.append(e)

    total3.append(saco1)        

# Se crea dataframe con datos ya estructurados con las columnas Nombre-Precio(S/.)-Cantidad

columnas=['Local','Nombre','Precio (S/.)','Cantidad']
Tottus=pd.DataFrame(data=total3,columns=columnas)
driver.close()
#Tottus
print("Datos de Tottus descargados")
################################################# PARA PLAZAVEA #################################################


# FUNCIONES QUE CREAMOS PARA UTILIZARLAS POSTERIORMENTE:

# PAQUETES UTILIZADOS
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import xml.etree.ElementTree as ET
import time
import pandas as pd
    
# Función que selecciona la pantalla de todos los productos en general e individualmente
def it():
    timeout=25 #tiempo de espera por pagina
    element_present = EC.presence_of_element_located((By.CLASS_NAME, 'showcase-grid'))
    WebDriverWait(browser, timeout).until(element_present) #Para esperar que cargue toda la pantalla con todos los productos
    elem_list = browser.find_element(
        By.CLASS_NAME, "showcase-grid") #Buscando todos los productos
    items = elem_list.find_elements(
        By.XPATH, '//div[@class="HA Showcase Showcase--food"]') #Seleccionando cada producto
    return items

# Función que almacena los valores que nos interesan de cada producto
def alm(): 
    table = [] # en donde se almacera toda la pagina
    for item in it():
        prov = [] # en donde se almacera los valores de un solo producto
        
        pagina="PlazaVea"
        
        prov.append(pagina)
        
        try:
            title = item.find_element(By.CLASS_NAME, 'Showcase__content').get_attribute("title")
            prov.append(title) #Seleccionando el titulo del producto
        except:
            title = "Sin info"
            prov.append(title)
            pass #En caso de que no seleccione, que pase.
        
        try:
            price = item.find_element(By.CLASS_NAME, "Showcase__salePrice").get_attribute("data-price")
            prov.append(price) #Seleccionando el precio del producto
        except:
            price = "Sin info"
            prov.append(price)
            pass #En caso de que no seleccione, que pase.
        
        try:
            cantidad = item.find_element(By.CLASS_NAME, "Showcase__units-reference").text
            prov.append(cantidad) #Seleccionando la cantidad del producto
        except:
            cantidad = "Sin info"
            prov.append(cantidad)
            pass #En caso de que no seleccione, que pase.
        
        
        table.append(prov) #almacena todos los productos de una sola pagina
    return table

#print("hecho") #Imprime hecho si todo corrio correctamente

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
options.add_argument('--incognito')


#x=str(input("Coloca el producto de interes:"))

browser = webdriver.Chrome(ubicacion,options = options)

#Buscar el producto

url = "https://www.plazavea.com.pe/search/?_query="+producto #Aqui colocamos los valores de la pagina y la categoria
browser.get(url)

WebDriverWait(browser, 50)\
    .until(EC.element_to_be_clickable((By.CLASS_NAME,
                                      'Header__dropdown__iconList__login__notification')))\
    .click()

time.sleep(2)

WebDriverWait(browser, 50)\
    .until(EC.element_to_be_clickable((By.CLASS_NAME,
                                      'CustomModal__close')))\
    .click()

elem_n = browser.find_element(
        By.CLASS_NAME, "pagination__nav") #Buscando todos los productos
itemx = elem_n.find_elements(
    By.CLASS_NAME, 'pagination__item') #Seleccionando cada producto

paginas = []

for item in itemx:
    paginas.append(item.text)

paginas2 = []
r=0
while r < len(paginas):
    if paginas[r].isdigit() == True:
        paginas2.append(paginas[r])
    r+=1

x=paginas2[len(paginas2)-1]
#print("Cantidad de pagina", x)
n=0

datos=[]

while n<=int(x):
    try:
        n+=1
        WebDriverWait(browser, 15)\
            .until(EC.element_to_be_clickable((By.XPATH,
                                              '/html/body/div[4]/main/div/div[2]/div/section[2]/div[2]/div/div[3]/div/div[4]/div/div/span['+str(n)+']')))\
            .click()
        k=alm()
        #print(len(k),x)
        datos.extend(k)
    except:
        break
#print(datos)

import pandas as pd

PlazaVea=pd.DataFrame(datos,columns = ['Local','Nombre','Precio (S/.)','Cantidad'])
browser.close()
#PlazaVea
print("Datos de PlazaVea descargados")

#COSAS POR MEJORAR:
# - QUE PASA SI COLOCAS CHOCOLATE, ES UN VALOR QUE TIENE MUCHAS PAGINAS, ES NECESARIO HACER UN IF PARA CUANDO SON MUCHAS PAGINAS
# - QUE PASA SI NO COLOCAS NADA, FALTA COLOCAR QUE NO SE COLOCO NINGUN RESULTADO
# - QUE PASA SI COLOCO ALGO QUE NO TIENE SENTIDO
# - ENCONTRAR UNA FORMA PARA QUE CARGUE TODOS LOS PRODUCTOS ANTES DE PASAR DE PAGINA


################################################# PARA JUNTAR LAS TABLAS #################################################


data=[]
df_marks = pd.DataFrame(data)
df_marks = df_marks.append(Metro)
df_marks = df_marks.append(PlazaVea)
df_marks = df_marks.append(Tottus)


################################################# PARA ENVIAR AL CORREO #################################################

# Envío de de los resultados al correo ingresado anteriormente (SIN COMPARAR)
from pretty_html_table import build_table
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import getpass

# Se establece un usuario y contraseña por defecto quien va a enviar el correo
# usuario: jossemen01@gmail.com
# contraseña: mxszphkwuhexmfue       <- contraseña generado para aplicaciones autorizadas en la cuenta gmail quien envía.

sender_address = 'jossemen01@gmail.com'
sender_pass = 'mxszphkwuhexmfue'
receiver_address = correo

# El cuerpo del mensaje puede ser html, json, texto puro
body = """
<html>
<head>
</head>

<body>
        {0}
</body>

</html>
""".format(build_table(df_marks, 'blue_light'))    # Se puede cambiar el estilo de la tabla 'blue_light'


message = MIMEMultipart("alternative")  # Tipos de mensaje multiparte como html
message['From'] = sender_address
message['To'] = receiver_address
message['Subject'] = 'Resultados de la comparación de producto en 3 localidades'   

message.attach(MIMEText(body, 'html'))   # plain -> texto puro(plano)   html-> formato html  json...
# Establece el tipo de correo y su puerto (587)
session = smtplib.SMTP('smtp.gmail.com', 587)
# Inicia sesión
session.starttls()
session.login(sender_address, sender_pass)
text = message.as_string()
# Envío de correo
session.sendmail(sender_address, receiver_address, text)
session.quit()

print('Los resultados de la comparación fueron enviados con éxito a su correo')

Ingrese el producto a comparar: avena
Ingrese su dirección de correo electrónico (ej: example@gmail) giankuway@gmail.com


<ipython-input-80-d456177563b6>:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ubicacion,options = options)


Datos de Metro descargados


<ipython-input-80-d456177563b6>:194: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ubicacion,options = options)


Datos de Tottus descargados


<ipython-input-80-d456177563b6>:360: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ubicacion,options = options)
